In [3]:
import psycopg2
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_columns', None) #show all columns
pd.set_option('max_colwidth',None)

In [4]:
def search_disease(name, download_as=''):
    temp = name.lower()
    for i in temp:
        if i > "z" or i < "a":
            assert "TypeError"
    del temp
    conn = psycopg2.connect(database="mimic", user="postgres",
                            password="pandora", host="127.0.0.1", port="5432")
    cursor = conn.cursor()
    cursor.execute("""select subject_id, hadm_id, icd9_code, seq_num, long_title 
    from (mimic.diagnoses_icd 
    left join mimic.d_icd_diagnoses using (icd9_code)) 
    where long_title like """ + "'%" + name + "%'")
    result = cursor.fetchall()
    if(len(result)!=0):
        colnames = [desc[0] for desc in cursor.description]
        columns = cursor.fetchall()
        cursor.close()
        conn.close()
        columns = pd.DataFrame(columns)
        result = pd.DataFrame(result)
        result.columns = colnames
    if(download_as!=''):
        result.to_csv(os.path.dirname(os.getcwd())+'/data/download data/' + download_as + '.csv', index=False)
    print('there are', len(result['long_title'].unique()), 'related diseases of', name, ':\n', 
          result['long_title'].unique())
    
    return result

In [19]:
search_disease(name = 'failure', download_as='')

there are 52 related diseases of failure :
 ['Other ovarian failure' 'Rheumatic heart failure (congestive)'
 'Malignant hypertensive heart disease without heart failure'
 'Malignant hypertensive heart disease with heart failure'
 'Benign hypertensive heart disease without heart failure'
 'Benign hypertensive heart disease with heart failure'
 'Unspecified hypertensive heart disease without heart failure'
 'Unspecified hypertensive heart disease with heart failure'
 'Hypertensive heart and chronic kidney disease, malignant, without heart failure and with chronic kidney disease stage I through stage IV, or unspecified'
 'Hypertensive heart and chronic kidney disease, malignant, with heart failure and with chronic kidney disease stage I through stage IV, or unspecified'
 'Hypertensive heart and chronic kidney disease, malignant, with heart failure and with chronic kidney disease stage V or end stage renal disease'
 'Hypertensive heart and chronic kidney disease, benign, with heart failure

,subject_id,hadm_id,icd9_code,seq_num,long_title
0,11149,188682,25639,4,Other ovarian failure
1,48297,103539,25639,23,Other ovarian failure
2,66437,171698,25639,8,Other ovarian failure
3,83,158569,39891,2,Rheumatic heart failure (congestive)
4,195,118936,39891,3,Rheumatic heart failure (congestive)
...,...,...,...,...,...
41676,28855,158506,E8744,13,"Mechanical failure of instrument or apparatus during aspiration of fluid or tissue, puncture, and catheterization"
41677,28212,190127,E8744,16,"Mechanical failure of instrument or apparatus during aspiration of fluid or tissue, puncture, and catheterization"
41678,43128,150895,E8744,31,"Mechanical failure of instrument or apparatus during aspiration of fluid or tissue, puncture, and catheterization"
41679,54177,114373,E8748,5,Mechanical failure of instrument or apparatus during other specified procedures
